# Person and Car Detection

This code illustrates the transfer learning apporach for person and car detection on given data.



1.   We will be using YOLOv5 model for our training. Alternatively we can use any detector model frameworks such as yolov3-v4, FasterRCNN or EfficientDets. But here for ease of training I've showcased the training with this model.
2.   Our aim is to fine-tune a model on the given 2239 images. We'll initialize weights using pre-trained coco models since coco model is alredy trained on person,car images, it'll be easier to increase accuracy.
3. This example is showed on Google Colab.



In [1]:
#clone the yolov5 repo.
!git clone https://github.com/ultralytics/yolov5.git



Cloning into 'yolov5'...
remote: Enumerating objects: 12125, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 12125 (delta 0), reused 0 (delta 0), pack-reused 12122
Receiving objects: 100% (12125/12125), 11.93 MiB | 25.89 MiB/s, done.
Resolving deltas: 100% (8395/8395), done.


In [2]:
#Mounting google drive to upload the data.
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


Before uploading the data, I've already segregated the data in train/ and val/ folders. 

Training Images: 2000 
Validation Images: 239 

Now, the task is to convert the JSON annotations to YOLO format for training.

# Data Preparation

In [3]:
#This cell, converts the input COCO json to YOLO Txt format with the
#required format of "category_id, x, y, w , h"
import os
import json
from tqdm import tqdm
import shutil


def make_folders(path="output"):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format: 
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format: 
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """
    
    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]
  
def convert_coco_json_to_yolo_txt(output_path, json_file):

    path = make_folders(output_path)

    with open(json_file) as f:
        json_data = json.load(f)

    # write _darknet.labels, which holds names of all classes (one class per line)
    label_file = os.path.join(output_path, "_darknet.labels")
    with open(label_file, "w") as f:
        for category in tqdm(json_data["categories"], desc="Categories"):
            category_name = category["name"]
            f.write(f"{category_name}\n")

    for image in tqdm(json_data["images"], desc="Annotation txt for each iamge"):
        img_id = image["id"]
        img_name = image["file_name"]
        img_width = image["width"]
        img_height = image["height"]

        anno_in_image = [anno for anno in json_data["annotations"] if anno["image_id"] == img_id]
        anno_txt = os.path.join(output_path, img_name.split(".")[0] + ".txt")
        with open(anno_txt, "w") as f:
            for anno in anno_in_image:
                category = anno["category_id"] -1 
                bbox_COCO = anno["bbox"]
                x, y, w, h = convert_bbox_coco2yolo(img_width, img_height, bbox_COCO)
                f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    print("Converting COCO Json to YOLO txt finished!")

Calling the function to convert all the data into TXT

In [ ]:
convert_coco_json_to_yolo_txt("/content/drive/MyDrive/person_car/data_train/all_labels/", "/content/drive/MyDrive/person_car/data_train/annotation_json/bbox-annotations.json")

Annotation txt for each iamge: 100%|██████████| 2239/2239 [00:28<00:00, 79.43it/s]


Converting COCO Json to YOLO txt finished!


In [ ]:
#moving labels of train/val into their respective folders.
def move_files(source_img_dir, source_label_dir, target_label_dir):
  for each_img in os.listdir(source_img_dir):
    # print(each_img)
    txt_name = each_img.split(".")[0]+ ".txt"
    source_file = os.path.join(source_label_dir, txt_name)
    target_file = os.path.join(target_label_dir, txt_name)
    print(txt_name)
    shutil.copy(source_file, target_file)


The below chunk of the code moves training labels and validation labels to their respective folders according to image names.

In [ ]:

ALL_LABELS = "/content/drive/MyDrive/person_car/data_train/all_labels/"
TRAIN_DIR = "/content/drive/MyDrive/person_car/data_train/train"
VAL_DIR = "/content/drive/MyDrive/person_car/data_train/val"

train_imgs = os.path.join(TRAIN_DIR, "images")
val_imgs = os.path.join(VAL_DIR, "images")  

train_label_dir = os.path.join(TRAIN_DIR, "labels")
val_label_dir = os.path.join(VAL_DIR, "labels")
if not os.path.exists(train_label_dir):
  os.mkdir(os.path.join(TRAIN_DIR, "labels"))
if not os.path.exists(val_label_dir):
  os.mkdir(os.path.join(VAL_DIR, "labels"))



move_files(train_imgs, ALL_LABELS, train_label_dir)
move_files(val_imgs, ALL_LABELS, val_label_dir)



Now for training, we need to create a .YAML file to configure the train/val paths. Here, created a custom.yaml with required information.

In [4]:
#creating a yaml for training 
import yaml
data = dict(
    names = ['person', 'car'],
    nc = 2,
    path= "/content/drive/MyDrive/person_car/data_train/",
    train="/content/drive/MyDrive/person_car/data_train/train/images",
    val="/content/drive/MyDrive/person_car/data_train/val/images"

)

with open('/content/yolov5/custom.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

# Start Training

We will train our model with input_size 416x416 for 10 epochs.

*   We're using pretrained model `yolov5m.pt` to initialize. If we want deeper network with high accuracy we can use higher models from here: https://github.com/ultralytics/yolov5#pretrained-checkpoints

*   We can also train on higher resolution i.e 608 to achieve better performance.


In [1]:
!python3 /content/yolov5/train.py --img 416 --batch 8 --epochs 10 --data /content/yolov5/custom.yaml --weights yolov5m.pt

train: weights=yolov5m.pt, cfg=, data=/content/yolov5/custom.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=8, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-235-g632559b Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, h